# SMU Logo bounding box detection
Design an algorithm to identify all images of SMU from a set of random images. A bounding box should be drawn around the SMU logo whenever it appears in an image.

When an SMU logo occurs in an image, a bounding box should be drawn around it. Logo identification will be evaluated by its F1 score.

### Setup
Install dependencies
```bash
pip install -r requirements.txt

```

In [1]:
%pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


Import packages

In [2]:
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from dotenv import load_dotenv
from manage_model import update_path, convert_bbox_format, get_f1
import os

Load environment variables using dotenv
- `API_KEY` - API Key for Roboflow dataset
- `DATA_VERSION` - Version of dataset used
- `PATH_TO_DATA` - Absolute file path to the "SMU-Logo-Detection-1" folder. See .env.example for example

In [3]:
# Project settings
load_dotenv()
api_key = os.getenv('API_KEY')
data_version = os.getenv('DATA_VERSION')
path_to_project = os.getenv("PATH_TO_PROJECT")

# If on SMU Violet, can't load env variables. Write manually.

path_to_data = f'{path_to_project}/tree-type-detection-{data_version}'
print(path_to_data)

D:/Disk D Documents/CS424/Project SMU Logo/tree-type-detection-17


Load dataset into workspace

In [4]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("tree-dataset-iftyz").project("tree-type-detection-9rfxy")
dataset = project.version(data_version).download("yolov8")
print(f'Data version: {data_version}')

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tree-type-detection-17 in yolov8:: 100%|██████████| 17548/17548 [00:13<00:00, 1286.65it/s]


Data version: 17


Update test, train and val values in data.yaml
This fixes a file not found bug

In [5]:
update_path(data_version, path_to_project)

### [Training](https://docs.ultralytics.com/modes/train/#train-settings)
Use a trained model, saved in `/saved_models`, or train a model.
A pre-trained [model](https://github.com/ultralytics/ultralytics?tab=readme-ov-file) `yolov8n.pt` from Ultralytics should be used. Tune hyper-params such as learning rate and epochs.

If there is file not found error, check the `FILE_TO_PATH` env variable, and make sure that you've updated the `data.yml` to the absolute file path.

In [5]:
# # Train model (Laptop)
# model = YOLO('yolov8n.pt') # pre-trained yolov8 nano
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10)

# # Train model (SMU GPU)
model = YOLO('yolov8s.pt')
results = model.train(data=f'{path_to_data}/data.yaml', epochs=16, device='0', patience = 2, amp=False)

New https://pypi.org/project/ultralytics/8.1.24 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.11.2 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:/Disk D Documents/CS424/Project SMU Logo/tree-type-detection-9/data.yaml, epochs=16, patience=2, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buff

In [8]:
# Use saved model
# model = YOLO('./saved_models/v8s_epoch10.pt')

After training, the model can be found in `/runs`. Save the `best.pt` model in the `/saved_models` folder.

### [Validation](https://docs.ultralytics.com/modes/val/#usage-examples)
Val mode is used for validating a YOLOv8 model after it has been trained. In this mode, the model is evaluated on a validation set to measure its accuracy and generalization performance. This mode can be used to tune the hyperparameters of the model to improve its performance.

In [6]:
validation_results = model.val(data=f'{path_to_data}/data.yaml', conf=0.25, iou=0.6, device='0')
'''
1. Epochs: Vary from 7, 15 etc
2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics
3. Re-annotate if have time (those with very small logos)


4. annotate the rest of the images
5. after everything is annotated, augmentation and final training
'''

Ultralytics YOLOv8.0.196  Python-3.11.2 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)


Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-9\valid\labels.cache... 764 images, 635 backgrounds, 0 corrupt: 100%|██████████| 764/764 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:12<00:00,  3.91it/s]
                   all        764        149      0.878      0.685      0.787      0.598
Speed: 0.5ms preprocess, 13.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val37


'\n1. Epochs: Vary from 7, 15 etc\n2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics\n3. Re-annotate if have time (those with very small logos)\n\n\n4. annotate the rest of the images\n5. after everything is annotated, augmentation and final training\n'

In [7]:
get_f1(validation_results.results_dict)

0.7693864029423825

### [Test](https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference)
Predict bounding boxes on test set

In [ ]:
# Predict
# prediction = model.predict(source=f"{path_to_data}/test/images", conf=0.25, iou=0.6)
prediction = model.predict(source=f"{path_to_data}/../image_folder", conf=0.25, iou=0.6)

image 1/260 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-7\test\images\IMG_8165_JPG.rf.fc2fd99a3964df0741ace08abf6c88fe.jpg: 640x640 (no detections), 36.0ms
image 2/260 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-7\test\images\IMG_8198_JPG.rf.038cdba42e193e190a2a86e0bf101202.jpg: 640x640 (no detections), 37.0ms
image 3/260 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-7\test\images\IMG_8201_JPG.rf.44990e562877606569c014837f94c8eb.jpg: 640x640 (no detections), 33.5ms
image 4/260 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-7\test\images\IMG_8235_JPG.rf.e23f44338a0daa3601c29617d29246c4.jpg: 640x640 (no detections), 29.5ms
image 5/260 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-7\test\images\IMG_8243_JPG.rf.219c3534c406c665023d57077d82e8ed.jpg: 640x640 (no detections), 30.0ms
image 6/260 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-7\test\images\IMG_8246_JPG.rf.794fe03c686479718a6d

Save images to `/predictions` folder if bounding box exists.

In [9]:
images = {}

if not os.path.exists(f'{path_to_data}/../predictions'):
    os.makedirs(f'{path_to_data}/../predictions')

for i in range(0, len(prediction)):
    if prediction[i].boxes.shape[0] >= 1:
        im_rgb = Image.fromarray(prediction[i].plot(line_width=1)[..., ::-1])
        images[i] = im_rgb  # RGB-order PIL image
        predicted_box = convert_bbox_format(prediction[i])

        # Write labels
        with open(f'{path_to_data}/../predictions/obj_est.txt', 'a') as f:
            f.write(', '.join(map(str, predicted_box)) + '\n')
        # Save images
        im_rgb.save(fp=f'{path_to_data}/../predictions/{predicted_box[0]}_labelled.jpg')

len(images)

49